In [2]:
import atexit
import threading
from math import atan2, pi
from os import makedirs
from time import sleep

import cv2
import ipywidgets.widgets as widgets
import numpy as np
import traitlets
from jetbot import Robot
from traitlets.config.configurable import SingletonConfigurable

import xbox as xbox_controller


JOYSTICK_DZ_THRESH: float = 0.15
RIGHT_TRIGGER_THRESH: float = 0.1
PRECISION: int = 3
IMAGE_WIDTH: int = 820
IMAGE_HEIGHT: int = 616
COLLECTION_TPS: int = 15

FRAMES_DIRECTORY: str = './test2'
FRAMES_START_INDEX: int = 0


class Camera(SingletonConfigurable):
    
    value = traitlets.Any()
    
    # config
    width = traitlets.Integer(default_value=IMAGE_WIDTH).tag(config=True)
    height = traitlets.Integer(default_value=IMAGE_HEIGHT).tag(config=True)
    fps = traitlets.Integer(default_value=21).tag(config=True)
    capture_width = traitlets.Integer(default_value=3280).tag(config=True)
    capture_height = traitlets.Integer(default_value=2464).tag(config=True)

    def __init__(self, *args, **kwargs):
        self.value = np.empty((self.height, self.width, 3), dtype=np.uint8)
        super(Camera, self).__init__(*args, **kwargs)

        try:
            self.cap = cv2.VideoCapture(self._gst_str(), cv2.CAP_GSTREAMER)

            re, image = self.cap.read()

            if not re:
                raise RuntimeError('Could not read image from camera.')

            self.value = image
            self.start()
        except:
            self.stop()
            raise RuntimeError(
                'Could not initialize camera.  Please see error trace.')

        atexit.register(self.stop)

    def _capture_frames(self):
        while True:
            re, image = self.cap.read()
            if re:
                self.value = image
            else:
                break
                
    def _gst_str(self):
        return 'nvarguscamerasrc ! video/x-raw(memory:NVMM), width=%d, height=%d, format=(string)NV12, framerate=(fraction)%d/1 ! nvvidconv ! video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! videoconvert ! appsink' % (
                self.capture_width, self.capture_height, self.fps, self.width, self.height)
    
    def start(self):
        if not self.cap.isOpened():
            self.cap.open(self._gst_str(), cv2.CAP_GSTREAMER)
        if not hasattr(self, 'thread') or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if hasattr(self, 'cap'):
            self.cap.release()
        if hasattr(self, 'thread'):
            self.thread.join()
            
    def restart(self):
        self.stop()
        self.start()


class MotorValues(traitlets.HasTraits):
    motor_speed = traitlets.Float()
    
    
class Collector(traitlets.HasTraits):
    def __init__(self, frames_directory: str, frame_index: int):
        self.frames_directory = frames_directory
        self.frame_index = frame_index
        self.should_shutdown = False
        self.should_record = False
        
        self.robot: Robot = Robot()
        print('')
        self.camera: Camera = Camera.instance()
        print('Camera initialized')
        self.image_latest = widgets.Image(format='png', width=IMAGE_WIDTH, height=IMAGE_HEIGHT)
        
        self.controls = {'left_motor': MotorValues(),
                         'right_motor': MotorValues()}
        self.controller = xbox_controller.Joystick()
        print('Controller initialized')
        self.ls_x = 0.0
        self.ls_y = 0.0
        self.right_trigger = 0.0
        self.steering_vector = 0.0

        # Dynamically link motor controller values to class attribute
        self.left_link = traitlets.dlink((self.controls['left_motor'], 'motor_speed'), (self.robot.left_motor, 'value'))
        self.right_link = traitlets.dlink((self.controls['right_motor'], 'motor_speed'), (self.robot.right_motor, 'value'))
        
        # Dynamically link camera output to class attribute
        self.camera_link = traitlets.dlink((self.camera, 'value'), (self.image_latest, 'value'), transform=lambda value: bytes(cv2.imencode('.png', value)[1]))
        
        # Display latest frame from camera
        display(self.image_latest)
        
        self.threads = {'movement': threading.Thread(name='thread_movement', target=self.monitor_status),
                       'camera': threading.Thread(name='thread_camera', target=self.record_frames)}
        
        for thread in self.threads.values():
            thread.start()
        
    def shutdown(self):
        print('Shutdown procedure initiated...')
        self.controller.close()
        self.left_link.unlink()
        self.right_link.unlink()
        self.camera_link.unlink()
        self.camera.stop()
        self.robot.stop()
        print('Shutdown complete')
        exit(0)
        
    def save_frame(self, steering_vector: float = 0):
        """Saves a frame into a file in the frames_directory."""
        file_path = f'{self.frames_directory}/{self.frame_index}-{steering_vector}.png'
        with open(file_path, 'wb') as f:
            f.write(self.image_latest.value)
            
    def calculate_steering_vector(self):
        return atan2(self.ls_y, self.ls_x) / pi
        
    def record_frames(self):
        while not self.should_shutdown:
            # Ignore values of 0 and when y is negative (do not record when reversing)
            if self.ls_y > 0 and self.right_trigger > 0:
                self.save_frame(self.steering_vector)
                self.frame_index += 1
            
            # Throttle number of frames saved
            sleep(1. / COLLECTION_TPS)
            
    
    def monitor_status(self):
        while not self.should_shutdown:
            # Handle left joystick axes positions: right and up are positive (after inverting y axis)
            self.ls_x = round(self.controller.leftX(), PRECISION)
            self.ls_y = round(self.controller.leftY(), PRECISION)
            
             # Handle right trigger position, normalize value
            self.right_trigger = round(self.controller.rightTrigger(), PRECISION)

            # Compensate for controller deadzone
            if abs(self.ls_x) < JOYSTICK_DZ_THRESH:
                self.ls_x = 0.0
            if abs(self.ls_y) < JOYSTICK_DZ_THRESH:
                self.ls_y = 0.0
            if self.right_trigger < RIGHT_TRIGGER_THRESH:
                self.right_trigger = 0.0
            
            # Disable the car from moving if it tries to make a sharp turn
            self.steering_vector = self.calculate_steering_vector()
            if self.steering_vector < 0.25 or self.steering_vector > 0.75:
                self.ls_x = 0.0
                self.ls_y = 0.0
                self.right_trigger = 0.0

            # Calculate intermediaries for tank control conversion
            v = (1 - abs(self.ls_x)) * (self.ls_y / 1) + self.ls_y
            w = (1 - abs(self.ls_y)) * (self.ls_x / 1) + self.ls_x

            # Translate intermediaries and apply right trigger speed multiplier to compute tank control values
            self.controls['left_motor'].motor_speed = (v + w) / 2 * self.right_trigger
            self.controls['right_motor'].motor_speed = (v - w) / 2 * self.right_trigger

                
if __name__ == '__main__':
    makedirs(FRAMES_DIRECTORY, exist_ok=True)
    try:
        collector = Collector(FRAMES_DIRECTORY, FRAMES_START_INDEX)
    except Exception as e:
        collector.shutdown()
        print(e)
        
    try:
        for thread in collector.threads.values():
            thread.join()
    except KeyboardInterrupt:
        collector.should_shutdown = True
        for thread in collector.threads.values():
            thread.join()
            
        collector.shutdown()


Camera initialized
Controller initialized


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x034\x00\x00\x02h\x08\x02\x00\x00\x00?+3\x18\x00\x00…

Shutdown procedure initiated...
Shutdown complete


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-a0261d72dc1b>", line 62, in _capture_frames
    re, image = self.cap.read()
cv2.error: /home/nvidia/build_opencv/opencv/modules/videoio/src/cap_gstreamer.cpp:447: error: (-2) GStreamer: unable to start pipeline
 in function icvStartPipeline




In [1]:
# Cell for manual robot shutdown
collector.left_link.unlink()
collector.right_link.unlink()
collector.camera_link.unlink()
collector.camera.stop()
collector.controller.close()
collector.robot.stop()

NameError: name 'collector' is not defined